In [1]:
!pip install replay-rec rs_datasets implicit -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.5/231.5 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.8/28.8 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import random
import warnings
warnings.simplefilter(action='ignore')

from implicit.bpr import BayesianPersonalizedRanking
import numpy as np
import pandas as pd
from scipy.sparse.linalg import svds
from replay.metrics import HitRate, NDCG, Coverage, Experiment
from replay.preprocessing.converter import CSRConverter
from replay.preprocessing.filters import MinCountFilter, LowRatingFilter
from replay.splitters import TimeSplitter
from rs_datasets import MovieLens

In [3]:
random.seed(0)
np.random.seed(0)

In [4]:
USER_COL = 'user_id'
ITEM_COL = 'item_id'
RATING_COL = 'rating'
TIMESTAMP = 'timestamp'

In [5]:
SEED = 0

In [6]:
data = MovieLens("1m")
ratings = data.ratings
ratings.head()

INFO:rs_datasets:Downloading ml-1m from grouplens...
5.93MB [00:00, 8.68MB/s]                            


,user_id,item_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


## Предобработка данных

В датасете содержатся только пользователи с более чем 20 просмотренными фильмами. Отфильтруем фильмы, у которых менее 20 просмотров.

In [7]:
ratings = MinCountFilter(num_entries=20, groupby_column=ITEM_COL).transform(ratings)

In [8]:
print(f'Количество уникальных пользователей: {ratings[USER_COL].nunique()}')
print(f'Количество уникальных фильмов: {ratings[ITEM_COL].nunique()}')

Количество уникальных пользователей: 6040
Количество уникальных фильмов: 3043


Переведем рейтинги в float формат для корректной работы модели SVD.

In [9]:
ratings[RATING_COL] = ratings[RATING_COL].astype(float)

## train-test split

Разделим данные по времени так, чтобы 20% взаимодействий оказалось в тестовой выборке. Исключим из тестовой выборки холодных пользователей и холодные объекты.

In [10]:
train, test = TimeSplitter(time_threshold=0.2,
                           query_column=USER_COL,
                           item_column=ITEM_COL,
                           drop_cold_users=True,
                           drop_cold_items=True).split(ratings)
train.shape, test.shape

((796392, 4), (103764, 4))

Поменяем индексы пользователей и фильмов так, чтобы в нумерации не было пропусков.

In [11]:
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

In [12]:
all_users = train[USER_COL].astype('category').cat.codes
all_items = train[ITEM_COL].astype('category').cat.codes

user_id2idx = dict(zip(train[USER_COL], all_users))
item_id2idx = dict(zip(train[ITEM_COL], all_items))

train[USER_COL] = train[USER_COL].map(user_id2idx)
train[ITEM_COL] = train[ITEM_COL].map(item_id2idx)
test[USER_COL] = test[USER_COL].map(user_id2idx)
test[ITEM_COL] = test[ITEM_COL].map(item_id2idx)

Будем оценивать качество только на фильмах, имеющих высокие рейтинги в тестовых данных.

In [13]:
test = LowRatingFilter(value=4, rating_column=RATING_COL).transform(test)
test.shape

(55988, 4)

# Models

Базовый класс для формирования разреженной матрицы рейтингов и получения предсказаний.

In [28]:
class BaseFactorizationModel:
    def __init__(
        self,
        user_col=USER_COL,
        item_col=ITEM_COL,
        rating_col=RATING_COL,
    ):
        self.user_col = user_col
        self.item_col = item_col
        self.rating_col = rating_col

    def get_rating_matrix(self, data):
        return CSRConverter(
            first_dim_column=self.user_col,
            second_dim_column=self.item_col,
            data_column=self.rating_col,
        ).transform(data)

    def predict(self, scores, rating_matrix=None, filter_seen=True, k=10):
        if filter_seen:
            scores += np.nan_to_num(rating_matrix.todense() * -np.inf)

        ind_part = np.argpartition(scores, -k)[:, -k:].copy()
        scores_not_sorted = np.take_along_axis(scores, ind_part, axis=1)
        ind_sorted = np.argsort(scores_not_sorted, axis=1)
        scores_sorted = np.sort(scores_not_sorted, axis=1)
        indices = np.take_along_axis(ind_part, ind_sorted, axis=1)

        preds = pd.DataFrame({
            self.user_col: range(scores.shape[0]),
            self.item_col: np.flip(indices, axis=1).tolist(),
            self.rating_col: np.flip(scores_sorted, axis=1).tolist()
            }).explode([self.item_col, self.rating_col])

        return preds

# PureSVD

In [29]:
class SVD(BaseFactorizationModel):
    def __init__(
        self, user_col=USER_COL, item_col=ITEM_COL, n_factors=20
        ):
        super().__init__(user_col, item_col)
        self.n_factors = n_factors


    def fit(self, data):
        self.rating_matrix = self.get_rating_matrix(data)
        user_matrix, singular_values, item_matrix = svds(
            A=self.rating_matrix, k=self.n_factors
            )
        user_matrix = user_matrix * np.sqrt(singular_values)
        item_matrix = item_matrix.T * np.sqrt(singular_values)

        self.user_matrix = user_matrix
        self.item_matrix = item_matrix
        self.scores = user_matrix @ item_matrix.T

Обучим модель.

In [30]:
svd_model = SVD()
svd_model.fit(train)

Получим предсказания.

In [31]:
svd_scores = svd_model.scores
preds_svd = svd_model.predict(svd_scores, svd_model.rating_matrix)

In [32]:
preds_svd.head()

,user_id,item_id,rating
0,0,900,3.12136
0,0,886,2.788202
0,0,954,2.31215
0,0,1175,2.054553
0,0,463,2.032025


Посмотрим на статистику предсказанных рейтингов.

In [33]:
svd_scores = svd_scores[svd_scores > -1e+308]
np.min(svd_scores), np.median(svd_scores), np.max(svd_scores)

(-2.9503594788295637, 0.015013272650002366, 7.563265415446935)

## BPRMF

Сформируем разреженную матрицу рейтингов.

In [34]:
base_model = BaseFactorizationModel()
rating_matrix = base_model.get_rating_matrix(train)

Обучим модель.

In [35]:
bprmf_model = BayesianPersonalizedRanking(factors=20,
                                          regularization=0.1,
                                          iterations=50,
                                          use_gpu=False,
                                          random_state=SEED)
bprmf_model.fit((rating_matrix).astype('double'))

  0%|          | 0/50 [00:00<?, ?it/s]

Получим обученные векторы пользователей и объектов и матрицу скоров для каждой пары пользователь-объект.



In [36]:
user_vecs = bprmf_model.user_factors
item_vecs = bprmf_model.item_factors
bprmf_scores = user_vecs.dot(item_vecs.T)

Получим предсказания.

In [37]:
preds_bprmf = base_model.predict(bprmf_scores, rating_matrix)

In [38]:
preds_bprmf.head(10)

,user_id,item_id,rating
0,0,2237,1.308272
0,0,886,1.268852
0,0,2701,1.267347
0,0,1175,1.237426
0,0,900,1.177181
0,0,2010,1.142285
0,0,1041,1.122397
0,0,599,1.093526
0,0,1044,1.090882
0,0,1581,1.062766


Посмотрим на статистику предсказанных рейтингов.

In [39]:
bprmf_scores = bprmf_scores[bprmf_scores > -1e+308]
np.min(bprmf_scores), np.median(bprmf_scores), np.max(bprmf_scores)

(-1.9340906, -0.16353786, 1.609175)

# Метрики

Вычислим метрики качества для top-10 предсказанных фильмов у каждого пользователя.

In [40]:
K = [10]
metrics = Experiment(
    [
        NDCG(K),
        HitRate(K),
        Coverage(K)
    ],
    test,
    train,
    query_column=USER_COL,
    item_column=ITEM_COL,
    rating_column=RATING_COL,
)

In [41]:
metrics.add_result("SVD", preds_svd)
metrics.add_result("BPRMF", preds_bprmf)
metrics.results

,NDCG@10,HitRate@10,Coverage@10
SVD,0.234868,0.673214,0.240053
BPRMF,0.196625,0.609821,0.217363


##  Векторы пользователей c похожими и различными вкусами для модели BPRMF

In [42]:
bprmf_model.similar_users(userid=0)

(array([   0, 3808, 4656, 2281, 3605, 1143, 3649, 4398, 3089, 4975],
       dtype=int32),
 array([0.99999994, 0.99779737, 0.99771315, 0.9969613 , 0.99693155,
        0.9968067 , 0.99668366, 0.99634326, 0.9961636 , 0.99600154],
       dtype=float32))

In [43]:
similar_user_ind = bprmf_model.similar_users(userid=0)[0][1]
similar_user_ind

3808

Сравним векторы похожих пользователей, полученные при обучении модели.

In [44]:
user_vecs[0]

array([-0.03370765, -0.13026533,  0.24720056,  0.25050166, -0.6674749 ,
        0.37531024, -0.21704794, -0.02306515, -0.42703503,  0.15378179,
       -0.35515144,  0.07848091,  0.20284937, -0.27284107,  0.06674348,
        0.3197844 , -0.18579057, -0.02776764,  0.336901  ,  0.16452175,
        1.        ], dtype=float32)

In [45]:
user_vecs[similar_user_ind]

array([-0.00977483, -0.13558356,  0.23829183,  0.30042687, -0.6779355 ,
        0.3865572 , -0.19379634, -0.01867422, -0.43006098,  0.1717387 ,
       -0.35512483,  0.05040321,  0.1799024 , -0.2496228 ,  0.10544998,
        0.3399111 , -0.16339071, -0.05511104,  0.34499985,  0.21086909,
        1.        ], dtype=float32)

Найдем наименее похожего пользователя.

In [46]:
dissimilar_user_ind = bprmf_model.similar_users(userid=0, N=train[ITEM_COL].nunique())[0][-1]
dissimilar_user_ind

3737

In [47]:
user_vecs[dissimilar_user_ind]

array([-2.5655495e-02, -5.3993262e-02, -1.2109503e-01,  1.0253253e-01,
        1.7816176e-01, -9.1325045e-03, -1.9895110e-02,  5.6956090e-02,
        1.9894029e-01,  4.6225479e-03,  3.8310204e-02, -2.1150422e-03,
       -5.4496668e-02,  2.5770569e-01,  1.6785175e-01, -5.1124521e-02,
        1.2717418e-01,  5.0666105e-02, -5.6884363e-02,  5.6324824e-04,
        1.0000000e+00], dtype=float32)

Посмотрим количество одинаковых фильмов в истории просмотров похожих пользователей.

In [48]:
user_items = set(train[train[USER_COL] == 0][ITEM_COL])
similar_user_items = set(train[train[USER_COL] == similar_user_ind][ITEM_COL])
dissimilar_user_items = set(train[train[USER_COL] == dissimilar_user_ind][ITEM_COL])

In [49]:
print(f'Пользователь с индексом 0 посмотрел {len(user_items)} фильма')
print(f'Похожий на него пользователь посмотрел {len(similar_user_items)} фильмов')
print(f'Количество фильмов, которые посмотрели оба пользователя: {len(user_items & similar_user_items)}')

Пользователь с индексом 0 посмотрел 64 фильма
Похожий на него пользователь посмотрел 81 фильмов
Количество фильмов, которые посмотрели оба пользователя: 19


In [50]:
print(f'Непохожий на пользователя с индексом 0 пользователь посмотрел {len(dissimilar_user_items)} фильма')
print(f'Количество фильмов, которые посмотрели оба пользователя: {len(user_items & dissimilar_user_items)}')

Непохожий на пользователя с индексом 0 пользователь посмотрел 384 фильма
Количество фильмов, которые посмотрели оба пользователя: 22
